# Ejecución de scripts como un trabajo de canalización

Una canalización permite agrupar varios pasos en un flujo de trabajo. Puede compilar una canalización con componentes. Cada componente refleja un script de Python que se va a ejecutar. Un componente se define en un archivo YAML que especifica el script y cómo ejecutarlo. 

## Antes de comenzar

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Creación de los scripts

Compilará una canalización con dos pasos:

1. **Preparar los datos**: corrija los datos que faltan y normalice los datos.
1. **Entrenamiento del modelo**: entrena un modelo de clasificación de árbol de decisión.

Ejecute las celdas siguientes para crear la carpeta **src** y los dos scripts.

In [ ]:
## Definición de los componentes

Para definir el componente que debe especificar:

- **Metadatos**: *nombre*, *nombre para mostrar*, *versión*, *descripción*, *tipo*, etc. Los metadatos ayudan a describir y gestionar el componente.
- **Interfaz**: *entradas* y *salidas*. Por ejemplo, un componente de entrenamiento de modelos tomará los datos de entrenamiento y la tasa de regularización como entrada, y generará un archivo de modelo entrenado como salida. 
- **Comando, código y entorno**: el *comando*, *código* y *entorno* para ejecutar el componente. "Command" es el comando shell para ejecutar el componente. Normalmente, el código hace referencia a un directorio de código fuente. El entorno puede ser un entorno de AzureML (mantenido o creado a medida), una imagen de Docker o un entorno de Conda.

Ejecute las celdas siguientes para crear un YAML para cada componente que quiera ejecutar como paso de canalización.

In [ ]:
## Cargar los componentes

Ahora que ha definido cada componente, puede cargar los componentes haciendo referencia a los archivos YAML. 

## Crear la canalización

Después de crear y cargar los componentes, puede compilar la canalización. Creará los dos componentes en una canalización. En primer lugar, querrá que se ejecute el componente `prep_data`. La salida del primer componente debe ser la entrada del segundo componente `train_decision_tree`, que entrenará el modelo.

La función `diabetes_classification` representa la canalización completa. La función espera una variable de entrada: `pipeline_job_input`. Durante la instalación se creó un recurso de datos. Usará el recurso de datos registrado como entrada de canalización. 

In [ ]:
Puede recuperar la configuración del trabajo de canalización imprimiendo el objeto `pipeline_job`:

In [ ]:
Puede cambiar cualquier parámetro de la configuración del trabajo de canalización haciendo referencia al parámetro y especificando el nuevo valor:

In [ ]:
## Enviar el trabajo de canalización

Por último, cuando haya compilado la canalización y configurado el trabajo de canalización para que se ejecute según sea necesario, puede enviar el trabajo de canalización:

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job